### Import

In [364]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import  OrdinalEncoder
from sklearn.ensemble import ExtraTreesClassifier

### Data Load

In [365]:
train = pd.read_csv('./train.csv').drop(columns=['ID'])
test = pd.read_csv('./test.csv').drop(columns=['ID'])

In [366]:
X = train.drop('임신 성공 여부', axis=1)
y = train['임신 성공 여부']

### Data Pre-processing

In [367]:
categorical_columns = [
    "시술 시기 코드",
    "시술 당시 나이",
    "시술 유형",
    "특정 시술 유형",
    "배란 자극 여부",
    "배란 유도 유형",
    "단일 배아 이식 여부",
    "착상 전 유전 검사 사용 여부",
    "착상 전 유전 진단 사용 여부",
    "남성 주 불임 원인",
    "남성 부 불임 원인",
    "여성 주 불임 원인",
    "여성 부 불임 원인",
    "부부 주 불임 원인",
    "부부 부 불임 원인",
    "불명확 불임 원인",
    "불임 원인 - 난관 질환",
    "불임 원인 - 남성 요인",
    "불임 원인 - 배란 장애",
    "불임 원인 - 여성 요인",
    "불임 원인 - 자궁경부 문제",
    "불임 원인 - 자궁내막증",
    "불임 원인 - 정자 농도",
    "불임 원인 - 정자 면역학적 요인",
    "불임 원인 - 정자 운동성",
    "불임 원인 - 정자 형태",
    "배아 생성 주요 이유",
    "총 시술 횟수",
    "클리닉 내 총 시술 횟수",
    "IVF 시술 횟수",
    "DI 시술 횟수",
    "총 임신 횟수",
    "IVF 임신 횟수",
    "DI 임신 횟수",
    "총 출산 횟수",
    "IVF 출산 횟수",
    "DI 출산 횟수",
    "난자 출처",
    "정자 출처",
    "난자 기증자 나이",
    "정자 기증자 나이",
    "동결 배아 사용 여부",
    "신선 배아 사용 여부",
    "기증 배아 사용 여부",
    "대리모 여부",
    "PGD 시술 여부",
    "PGS 시술 여부"
]

In [368]:
# 카테고리형 컬럼들을 문자열로 변환
for col in categorical_columns:
    X[col] = X[col].astype(str)
    test[col] = test[col].astype(str)

In [369]:
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

X_train_encoded = X.copy()
X_train_encoded[categorical_columns] = ordinal_encoder.fit_transform(X[categorical_columns])

X_test_encoded = test.copy()
X_test_encoded[categorical_columns] = ordinal_encoder.transform(test[categorical_columns])

In [370]:
numeric_columns = [
    "임신 시도 또는 마지막 임신 경과 연수",
    "총 생성 배아 수",
    "미세주입된 난자 수",
    "미세주입에서 생성된 배아 수",
    "이식된 배아 수",
    "미세주입 배아 이식 수",
    "저장된 배아 수",
    "미세주입 후 저장된 배아 수",
    "해동된 배아 수",
    "해동 난자 수",
    "수집된 신선 난자 수",
    "저장된 신선 난자 수",
    "혼합된 난자 수",
    "파트너 정자와 혼합된 난자 수",
    "기증자 정자와 혼합된 난자 수",
    "난자 채취 경과일",
    "난자 해동 경과일",
    "난자 혼합 경과일",
    "배아 이식 경과일",
    "배아 해동 경과일"
]

In [371]:
from sklearn.impute import SimpleImputer
import numpy as np
import pandas as pd

# 1️⃣ 결측 여부 Feature 추가
for col in numeric_columns:
    X_train_encoded[col + '_missing'] = X_train_encoded[col].isna().astype(int)
    X_test_encoded[col + '_missing'] = X_test_encoded[col].isna().astype(int)

# zero_imputer = SimpleImputer(strategy="constant", fill_value=0)
# X_train_encoded[numeric_columns] = zero_imputer.fit_transform(X_train_encoded[numeric_columns])
# X_test_encoded[numeric_columns] = zero_imputer.transform(X_test_encoded[numeric_columns])

from numpy import log1p
# 🔹 로그 변환 적용 (Skewed Data Handling)
skewed_cols = ['총 생성 배아 수', '수집된 신선 난자 수', '저장된 배아 수', '미세주입된 난자 수']

for col in skewed_cols:
    X_train_encoded[col + '_log'] = log1p(X_train_encoded[col])
    X_test_encoded[col + '_log'] = log1p(X_test_encoded[col])

# 3️⃣ 80% 이상 결측치가 있는 컬럼 제거
missing_ratio = X_train_encoded.isnull().mean()
high_missing_columns = missing_ratio[missing_ratio > 0.8].index.tolist()
X_train_encoded.drop(columns=high_missing_columns, inplace=True)
X_test_encoded.drop(columns=high_missing_columns, inplace=True)

# 4️⃣ Feature Engineering (특성 추가)
X_train_encoded['배아_생성_효율'] = X_train_encoded['총 생성 배아 수'] / (X_train_encoded['수집된 신선 난자 수'] + 1)
X_test_encoded['배아_생성_효율'] = X_test_encoded['총 생성 배아 수'] / (X_test_encoded['수집된 신선 난자 수'] + 1)

X_train_encoded['배아_저장_비율'] = X_train_encoded['저장된 배아 수'] / (X_train_encoded['총 생성 배아 수'] + 1)
X_test_encoded['배아_저장_비율'] = X_test_encoded['저장된 배아 수'] / (X_test_encoded['총 생성 배아 수'] + 1)

X_train_encoded['난자_배아_비율'] = X_train_encoded['미세주입된 난자 수'] / (X_train_encoded['총 생성 배아 수'] + 1)
X_test_encoded['난자_배아_비율'] = X_test_encoded['미세주입된 난자 수'] / (X_test_encoded['총 생성 배아 수'] + 1)


# 5️⃣ 이상치 처리 (Clip 적용)
for col in ['총 생성 배아 수', '수집된 신선 난자 수', '저장된 배아 수']:
    X_train_encoded[col] = X_train_encoded[col].clip(lower=1, upper=50)
    X_test_encoded[col] = X_test_encoded[col].clip(lower=1, upper=50)

# 6️⃣ Feature Scaling 제거 (트리 모델은 불필요)

import seaborn as sns
import matplotlib.pyplot as plt

# 상관 행렬 계산
corr_matrix = X_train_encoded.corr()

# 높은 상관관계(절대값 0.9 이상)를 가지는 변수 찾기
high_corr_features = set()
for i in range(len(corr_matrix.columns)):
    for j in range(i):
        if abs(corr_matrix.iloc[i, j]) > 0.9:
            colname = corr_matrix.columns[i]
            high_corr_features.add(colname)

print("높은 상관관계를 가지는 Feature:", high_corr_features)

# 제거
X_train_encoded.drop(columns=high_corr_features, inplace=True)
X_test_encoded.drop(columns=high_corr_features, inplace=True)


높은 상관관계를 가지는 Feature: {'기증 배아 사용 여부', '부부 주 불임 원인', 'IVF 임신 횟수', '총 생성 배아 수_missing', '배아 해동 경과일_missing', '대리모 여부', '혼합된 난자 수_missing', '파트너 정자와 혼합된 난자 수_missing', '저장된 배아 수_missing', '난자 채취 경과일_missing', '이식된 배아 수_missing', '미세주입된 난자 수_log', '미세주입 배아 이식 수_missing', '기증자 정자와 혼합된 난자 수_missing', '해동된 배아 수_missing', '해동 난자 수_missing', '배란 유도 유형', '미세주입에서 생성된 배아 수', 'IVF 출산 횟수', '저장된 신선 난자 수_missing', '미세주입 후 저장된 배아 수_missing', '파트너 정자와 혼합된 난자 수', 'IVF 시술 횟수', '미세주입된 난자 수_missing', '미세주입에서 생성된 배아 수_missing', '수집된 신선 난자 수_missing', '착상 전 유전 진단 사용 여부'}


### 검증

In [372]:
import numpy as np
import pandas as pd
from sklearn.ensemble import ExtraTreesClassifier 
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score 
from tqdm import tqdm 

# 모델 정의
model = ExtraTreesClassifier(
    n_estimators=300,  # 트리 개수 증가
    max_depth=15,  # 트리 깊이 제한
    min_samples_split=5,  # 노드 분할 최소 샘플 수 증가
    min_samples_leaf=2,  # 최소 리프 노드 크기 증가
    bootstrap=True,  # 부트스트랩 샘플링 적용
    random_state=42,
    n_jobs=-1
)

# 교차 검증 설정 (5-Fold)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 진행도 표시를 위한 tqdm 적용
cv_scores = []
for fold, (train_idx, val_idx) in enumerate(tqdm(cv.split(X_train_encoded, y), desc="Cross Validation Progress", total=5)):
    X_train_fold, X_val_fold = X_train_encoded.iloc[train_idx], X_train_encoded.iloc[val_idx]
    y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]
    
    model.fit(X_train_fold, y_train_fold)  # 모델 학습
    y_val_pred = model.predict_proba(X_val_fold)[:, 1]  # ROC-AUC 계산을 위해 확률값 추출
    
    auc_score = roc_auc_score(y_val_fold, y_val_pred)
    cv_scores.append(auc_score)
    
    print(f"Fold {fold+1}: ROC-AUC = {auc_score:.4f}")

# 최종 결과 출력
print(f"\n✅ 5-Fold ROC-AUC 점수 평균: {np.mean(cv_scores):.4f}")
print(f"각 Fold 점수: {cv_scores}")

Cross Validation Progress:  20%|██        | 1/5 [00:02<00:11,  2.97s/it]

Fold 1: ROC-AUC = 0.7309


Cross Validation Progress:  40%|████      | 2/5 [00:06<00:09,  3.05s/it]

Fold 2: ROC-AUC = 0.7358


Cross Validation Progress:  60%|██████    | 3/5 [00:08<00:05,  2.96s/it]

Fold 3: ROC-AUC = 0.7342


Cross Validation Progress:  80%|████████  | 4/5 [00:11<00:02,  2.93s/it]

Fold 4: ROC-AUC = 0.7322


Cross Validation Progress: 100%|██████████| 5/5 [00:14<00:00,  2.93s/it]

Fold 5: ROC-AUC = 0.7328

✅ 5-Fold ROC-AUC 점수 평균: 0.7332
각 Fold 점수: [np.float64(0.730948288210858), np.float64(0.735799846567335), np.float64(0.7342454341868925), np.float64(0.7322010395605417), np.float64(0.7328249690294699)]


### Train

In [373]:
# model = ExtraTreesClassifier(random_state=42)

model.fit(X_train_encoded, y)

ExtraTreesClassifier(bootstrap=True, max_depth=15, min_samples_leaf=2,
                     min_samples_split=5, n_estimators=300, n_jobs=-1,
                     random_state=42)

### Predict

In [374]:
pred_proba = model.predict_proba(X_test_encoded)[:, 1]

### Submission

In [375]:
sample_submission = pd.read_csv('./sample_submission.csv')
sample_submission['probability'] = pred_proba

In [376]:
sample_submission.to_csv('./baseline_submit.csv', index=False)